# Identification of Best Model 


## Setup 

In [ ]:
import pickle
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba

import seaborn as sns

import Utilities
import importlib
importlib.reload(Utilities)

import MLUtilities
importlib.reload(MLUtilities)

from sklearn.pipeline import Pipeline


# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler, PowerTransformer, RobustScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import  OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score, precision_score

from sklearn.ensemble import RandomForestClassifier


# from hyperopt.pyll import scope
# from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK

from hyperopt import hp,Trials,fmin,tpe,STATUS_OK

import xgboost as xgb

from imblearn.over_sampling import SMOTENC



# import sys
# import os
# import pickle

# import pandas as pd
# import numpy as np
# import seaborn as sns
# #import statsmodels.formula.api as smf
# #from statsmodels.stats.anova import anova_lm


# import re
# import matplotlib.pyplot as plt
# from openpyxl import load_workbook
# from pandasql import sqldf
# import sqlalchemy
# import pyodbc

# import dfply as dfp
# from dfply import *

# import datetime
# from datetime import *

# import pyodbc
# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
# from sklearn.metrics         import balanced_accuracy_score, precision_score, classification_report
# from sklearn.metrics         import recall_score, f1_score, make_scorer, cohen_kappa_score

# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline
# import sklearn

# from sklearn import model_selection
# from sklearn.model_selection import cross_val_score

# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn import svm
# from sklearn import tree
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split


# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier

# import forestci as fci

# from sklearn.linear_model import BayesianRidge
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer, PowerTransformer
# from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score, precision_score
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import cross_val_predict
# from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
# #from statsmodels.stats.outliers_influence import variance_inflation_factor
# #from statsmodels.tools.tools import add_constant
# import xgboost as xgb
# from imblearn.over_sampling import SMOTENC
# n_est=55

# from hyperopt.pyll import scope
# from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK

# pd.set_option('display.max_columns', None)


# import Utilities
# import importlib
# importlib.reload(Utilities)

# import MLUtilities
# importlib.reload(MLUtilities)


plt.style.use('./CEIH.mplstyle')
color_dict = {0: to_rgba('#32C0D2', 1),
                1: to_rgba("#E0B165", 1),}

cmap_blended = sns.blend_palette(["#ADE6ED","#70D3E0","#32C0D2","#289AA8","#307078"], as_cmap=True)


# import math
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1],idx-1
    else:
        return array[idx],idx
    

    
OtherValues = {'TriageCategory_2':'Triage Category 2', 'SoBFlag': 'Shortness of Breath', 'TriageCategory_4':'Triage Category 4',
               'TriageCategory_1':'Triage Category 1', 'TriageCategory_5':'Triage Category 5', 'TriageCategory_3':'Triage Category 3',
             'FirstGCSScoreAdult':'Glasgow Coma Scale',
               'FirstUrinalysisBlood': 'Urinalysis Blood', 'IndigenousStatusDescription_Not Aboriginal-TSI':  'Non Indiginous',
               'IndigenousStatusDescription_Not Stated':  'Indiginous Status not stated', 'FirstUrinalysisLeukocytes':'Urinalysis Leukocytes',
               'FirstPulseRateBPM':'Pulse Rate', 'FirstRespiration':'Respiration Rate', 'FirstSpO2':'O2 Saturation (%)',  'Gender_Male':'Sex - Male',
               'FirstLevelofConsciousness':'Level of Consciousness', 'IndigenousStatusDescription_Aboriginal and TSI':'Aboriginal and TSI',
               'IndigenousStatusDescription_TSI':'TSI','Gender_Indeterminate':'Sex - Indeterminate', 'Gender_Unknown':'Sex - Unknown',
               'FirstO2Flow':'O2 Flow ','GENDERCODE_Male':'Sex - Male','GENDERCODE_Unknown':'Sex - Unknown',
               'GENDERCODE_Indeterminate':'Sex - Indeterminate'
               }

plotSettings = {
        #  ( bins, xmin, xmax, log/linear)
        'AGEONADMISSION':(45,19.5,109.5,'linear','Age (y)', False, 0,0),
        
        
        'FirstBloodGlucose':(50,0,100,'log','Blood Glucose [mmol/L]', False,0, 0),
        'FirstTemperatureDegreesC':(50,30,45,'log', r"Temperature [$^\circ$C]", True,35.5,38.1),
                'FirstWeightKg':(50,50,150,'log','Weight [kg]', False, 0,0),

        
        'FirstPainAssessment': (11,-0.5,10.5,'log','Pain Assessment', False, 8,10),
        'FirstBPSystolic': (50,50,250,'log', 'BP Systolic [mm Hg]',True, 100,170 ),
        'FirstBPDiastolic': (50,0,200,'log', 'BP Diastolic [mm Hg]', False,0,0),
        'FirstEstimatedGlomerularFiltrationRate': (50,0,100,'log',r"Estimated Glomerular Filtration Rate [mL/min/1.73m$^{2}$]", True, 60, 100),
        'FirstCreatinine': (50,0,800,'log',r"Creatinine - Serum [$\mu$mol/L]", True, 45, 110),
        'FirstAlbumin': (60,0,60,'log','Albumin  Level [g/L]', True, 30, 48),
        'FirstTotalBilirubin': (60,0,100,'log',r"Total Bilirubin Level [$\mu$mol/L]", True, 2, 24 ),
        'FirstAlkalinePhosphatase': (60,0,800,'log','Alkaline Phosphatase Level [U/L]',True, 30,110),
        'FirstAlanineAminotransferase': (60,0,700,'log','Alanine Aminotransferase Level [U/L]', True,0,55),
        'FirstAspartateAminotransferase': (60,0,700,'log','Aspartate Aminotransferase Level [U/L]', True, 0,45),
        'FirstGammaGlutamylTransferase': (60,0,700,'log','Gamma Glutamyl Transferase Level [U/L]', True, 0, 60),
        'FirstLactateDehydrogenase': (60,0,1200,'log','Lactate Dehydrogenase [U/L]', True, 120, 250),
        'FirstHaemoglobin': (50,10, 220,'log','Haemoglobin [g/L]', True, 115, 175),
        'FirstWhiteCellCount': (50,0, 50,'log',r"White Cell Count [$\times 10^{9}$/L]",True, 4,11),
        'FirstPlateletCount': (50,0, 1000,'log',r"Platelet Count [$\times 10^{9}$/L]",True,150,500),
        'FirstNeutrophils': (50,0, 50,'log',r"Absolute Neutrophil Count [$\times 10^{9}$/L]",True,1.80,7.50),
        'FirstDDimer': (40,0, 20,'log',r"D-Dimer [mg/L]",True,0,0.79),
        'FirstCreactiveprotein': (50,0, 600,'log',r"C-Reactive Protein [mg/L]",True, 0,8),
        'FirstTroponinT': (50,0, 600,'log',r"Troponin T Level [mg/L]",True,0,16),
        'FirstNTproBNP': (50,0, 40000,'log',r"NT-pro Brain Natriuretic Peptide [mg/L]",True,0,124),
        
        'FirstAnionGapVenous': (50,0, 50,'log',r"Anion Gap Venous [mmol/L]", True, 7, 17),
        'FirstAnionGapArterial': (50,0, 50,'log',r"Anion Gap Arterial [mmol/L]",  True, 7, 17),
        'FirstBaseExcessVenous': (50,-30, 30,'log',r"Base Excess Venous [mmol/L]", True, -3, 3),
        'FirstBaseExcessArterial': (50,-30, 30,'log',r"Base Excess Arterial [mmol/L]", True, -3, 3),
        'FirstBilirubinVenous': (60,0, 60,'log',r"Bilirubin Venous [$\mu$mol/L]", True, 2,24),
        'FirstBilirubinArterial': (60,0, 60,'log',r"Bilirubin Arterial [$\mu$mol/L]", True, 2,24),
        'FirstCarboxyhaemoglobinVenous': (50,0, 20,'log',r"Carboxyhaemoglobin Venous [%]", True, 0.3, 1.8),
        'FirstCarboxyhaemoglobinArterial': (50,0, 20,'log',r"Carboxyhaemoglobin Arterial [%]", True, 0.3, 1.8),
        'FirstChlorideDirectVenous': (50,50, 150,'log',r"Chloride Direct Venous [mmol/L]",False, 100,109),
        'FirstChlorideDirectArterial': (50,50, 150,'log',r"Chloride Direct Arterial [mmol/L]", True, 100,109),
        'FirstCreatinineVenous':(50,0,500,'log',r"Creatinine Venous [$\mu$mol/L]", True, 50, 120),
        'FirstCreatinineArterial':(50,0,500,'log',r"Creatinine Arterial [$\mu$mol/L]", True, 50,120),
        'FirstGlucoseVenous':(50,0,30,'log',r"Glucose  Venous [mmol/L]", False,0,0),
        'FirstGlucoseArterial':(50,0,30,'log',r"Glucose  Arterial [mmol/L]", True,2.6,5.6),
        'FirstIonised Calcium Venous':(50,0,2,'log',r"Ionised Calcium Venous [mmol/L]", True, 1.1, 1.3),
        'FirstIonised Calcium Arterial':(50,0,2,'log',r"Ionised Calcium Arterial [mmol/L]", True, 1.1, 1.3),
        'FirstLactateVenous':(50,0,30,'log',r"Lactate Venous [mmol/L]", True, 0.2, 2.0),
        'FirstLactateArterial':(50,0,30,'log',r"Lactate Arterial [mmol/L]", True, 0.2, 2.0),
        'FirstMethaemoglobinVenous': (20,0, 3,'log',r"Methaemoglobin Venous [%]", True, 0.4, 1.2),
        'FirstMethaemoglobinArterial': (20,0, 3,'log',r"Methaemoglobin Arterial [%]", True, 0.2,0.6),
        'FirstOxygenSaturationVenous': (50,0, 100,'log',r"Oxygen Saturation Venous [%]", False, 0,0),
        'FirstOxygenSaturationArterial': (50,0, 100,'log',r"Oxygen Saturation Arterial [%]", True,95, 99),
        'FirstOxyhaemoglobinVenous': (50,0, 100,'log',r"Oxyhaemoglobin Venous [%]", False, 0,0),
        'FirstOxyhaemoglobinArterial': (50,0, 100,'log',r"Oxyhaemoglobin Arterial [%]", False, 0,0),
        'FirstReducedHaemoglobinVenous': (50,0, 100,'log',r"Reduced Haemoglobin Venous [%]", False, 0,0),
        'FirstReducedHaemoglobinArterial': (50,0, 100,'log',r"Reduced Haemoglobin Arterial [%]", False, 0,0),
        'FirstTotalHaemoglobinVenous': (50,10, 220,'log','Total Haemoglobin Venous [g/L]', True, 115,180),
        'FirstTotalHaemoglobinArterial': (50,10, 220,'log','Total Haemoglobin Arterial [g/L]', True, 115,180), 
        'FirstpCO2Venous': (50,0,150,'log', 'pCO2 Venous [mm Hg]', True,41,51),
        'FirstpCO2Arterial': (50,0,150,'log', 'pCO2 Arterial [mm Hg]', True, 35, 45 ),
        'FirstpO2Venous': (50,0,200,'log', 'pO2 Venous [mm Hg]', True, 25,40),
        'FirstpO2Arterial': (50,0,200,'log', 'pO2 Arterial [mm Hg]', True,67, 108),
        'FirstpHVenous': (50,6.8,7.8,'log', 'pH Venous', True, 7.32,7.42),
        'FirstpHArterial': (50,6.8,7.8,'log', 'pH Arterial', True, 7.36, 7.44),
        'FirstPotassiumDirectVenous':(50,0,10,'log',r"Potassium Direct Venous [mmol/L]", False, 0,0),
        'FirstPotassiumDirectArterial':(50,0,10,'log',r"Potassium Direct Arterial [mmol/L]", True, 3.1, 4.2),
        'FirstSodiumDirectVenous':(50,100,180,'log',r"Sodium Direct Venous [mmol/L]", False, 0,0),
        'FirstSodiumDirectArterial':(50,100,180,'log',r"Sodium Direct Arterial [mmol/L]", True, 137, 145),
        
        'FirstO2Flow':(20,0,20,'log',r"O$_{2}$ Flow", True,0,5),
        'FirstPulseRateBPM':(50,0,200,'log','Pulse Rate',True,60,100),
        'FirstSpO2':(50,0,100,'log',r"O$_{2}$ Saturation [%]",True,94,100),
         'FirstRespiration':(20,0,40,'log','Respiration Rate',True,10,21),
         'FirstLevelofConsciousness':(4,-0.5,3.5,'log',"Level of Consciousness",True,-0.5,2),
        'FirstGCSScoreAdult':(16,-0.5,15.5,'log','Glasgow Coma Scale',False,0,0)
        
        }


## Load and Setup Verification data

In [ ]:


# start_date =  '2023-01-01'  #dates[Facility][0]
# end_date   =  '2024-01-01' #dates[Facility][1]

# select_start_date = '2023-01-01'

# DataReasons = "Training"


start_date =  '2024-01-01'  #dates[Facility][0]
end_date   =  '2024-08-01'   #'2024-04-01' #dates[Facility][1]

select_start_date = '2023-01-01'


DataReasons = "Verification"
# clinics=Utilities.clinics(Facilit

Data_Storage_File = 'DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_{}_{}_{}_Stage_01.pkl'.format(start_date,end_date,DataReasons)
print(Data_Storage_File)


with open(Data_Storage_File, 'rb') as file:
    Emergency_IPInfo_Complete_Numeric_tmp = pd.read_pickle(file)

In [5]:
Data_Storage_File = 'DataFiles\Indigenous_EDI_list.pkl'   #.format(start_date,end_date,DataReasons)
print(Data_Storage_File)

with open(Data_Storage_File, 'rb') as file:
    indiginous_client_list = pd.read_pickle(file)

DataFiles\Indigenous_EDI_list.pkl


In [6]:
Emergency_IPInfo_Complete_Numeric_tmp['IndiginousFlag'] = 0  # np.where((Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isna()|Emergency_IPInfo_Complete_Numeric.IndiginousStatus=='Not Aboriginal-TSI'),0,1)
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.IndiginousStatus.isin(['Aboriginal','Aboriginal and TSI', 'TSI']),'IndiginousFlag'] = 1

Emergency_IPInfo_Complete_Numeric_tmp['IndigenousEDIFlag'] = 0  # np.where((Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isna()|Emergency_IPInfo_Complete_Numeric.IndiginousStatus=='Not Aboriginal-TSI'),0,1)
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.CLIENTGUID.isin(indiginous_client_list),'IndigenousEDIFlag'] = 1

In [7]:
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(0,17),'age_range']='0-17'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(18,33),'age_range']='18-33'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(34,48),'age_range']='34-48'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(49,64),'age_range']='49-64'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(65,78),'age_range']='65-78'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(79,98),'age_range']='79-98'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(98,120),'age_range']='98+'
Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION.between(121,200),'age_range']='Unknown'

Remove under 18s

In [8]:
Emergency_IPInfo_Complete_Numeric = Emergency_IPInfo_Complete_Numeric_tmp.loc[Emergency_IPInfo_Complete_Numeric_tmp.AGEONADMISSION>17].copy()

## Define DataSet and Diagnosis 

In [9]:
# Sepsis HeartFailure PE Pneumonia COPD UTI
# SepsisPneumonia
Diagnosis =   "Sepsis"
# Diagnosis = 'SepsisPneumonia'
# Diagnosis = 'Admission'
# Diagnosis = 'GenMed'

DiagnosisString=Diagnosis+'Flag'

PlotString = 'Sepsis Pathway'


# PlotString = 'Admitted'

# DataSet1 All observations 
# DataSet2 Not including Oxygen Flow 
# Dataset3  Dataset2 plus age and gender
#  Dataset4 3 plus restriction on TC to 3 4 and 5 
#  Dataset 3 with LoS > 6 hours. 



DataSet = 'DataSet60'

In [10]:


GenMedList = ['LMH-LGMED','LMH-LAMU',
              "MPH-MGMED", 
              "FMC-GENMED",
              "NHS-GENMED",
              "RAH-GENMED",
              "QEH-GENMED", 'QEH-GM-AMU'
              ]
# DiagnosisString='SepsisFlag'

if Diagnosis == "SepsisPneumonia":
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.SepsisFlag==1)|(Emergency_IPInfo_Complete_Numeric.PneumoniaFlag==1), DiagnosisString] = 1

if Diagnosis == 'Admission':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric['LOSHOUR']>24), DiagnosisString] = 1
    
    if DataSet == 'DataSet5':
        Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
        Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric['LOSHOUR']>6), DiagnosisString] = 1


if Diagnosis == 'GenMed':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    for g in GenMedList:
        Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.GROUPCODE.str.contains(g))  & (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1
   


print("Diagnosis being investigated = {}".format(DiagnosisString))
print("Total Number of Cases = {}".format(len(Emergency_IPInfo_Complete_Numeric.index)))
print("Number of {} Cases ".format(DiagnosisString),len(Emergency_IPInfo_Complete_Numeric[Emergency_IPInfo_Complete_Numeric[DiagnosisString]==1]))

print("\n\n*** Indigenous ***")
print("Total Number of Indiginous Cases = {}".format(len(Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.IndiginousFlag==1].index)))
print("Number of {} Cases ".format(DiagnosisString),len(Emergency_IPInfo_Complete_Numeric[(Emergency_IPInfo_Complete_Numeric[DiagnosisString]==1) & (Emergency_IPInfo_Complete_Numeric.IndiginousFlag==1)]))

print("\n\nTotal Number of EDI Indigenous Cases = {}".format(len(Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.IndigenousEDIFlag==1].index)))
print("Number of {} Cases ".format(DiagnosisString),len(Emergency_IPInfo_Complete_Numeric[(Emergency_IPInfo_Complete_Numeric[DiagnosisString]==1) & (Emergency_IPInfo_Complete_Numeric.IndigenousEDIFlag==1)]))


Diagnosis being investigated = SepsisFlag
Total Number of Cases = 164320
Number of SepsisFlag Cases  1618


*** Indigenous ***
Total Number of Indiginous Cases = 8440
Number of SepsisFlag Cases  71


Total Number of EDI Indigenous Cases = 7273
Number of SepsisFlag Cases  70


In [11]:
if DataSet == 'DataSet4':
    df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.TRIAGE_CATEGORY>2].copy()
else:
    df = Emergency_IPInfo_Complete_Numeric.copy()
df.name='{} diagnosis detected from full Diagnosis list, updated'.format(Diagnosis)

df = df.loc[:,~df.columns.duplicated()] # remove duplicate columns because I am a doofus

importlib.reload(MLUtilities)
df = MLUtilities.setDefaults(df)


df['FirstLactateBG'] = np.where(df['FirstLactateArterialDtm'] < df ['FirstLactateVenousDtm'], df['FirstLactateArterial'], df['FirstLactateVenous'])
df['FirstBaseExcessBG'] = np.where(df['FirstBaseExcessArterialDtm'] < df ['FirstBaseExcessVenousDtm'], df['FirstBaseExcessArterial'], df['FirstBaseExcessVenous'])

df['PrimaryDiagnosisCode'] = df['DIAGNOSIS_LIST'].str.split(' ').str[0]



AnalysisVariable =['FirstBPSystolic', 'FirstLevelofConsciousness',
       'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2',
       'FirstTemperatureDegreesC', 'FirstO2Flow',
       DiagnosisString]

if DataSet == 'DataSet3':
    AnalysisVariable.append('GENDERCODE')
    AnalysisVariable.append('AGEONADMISSION')
elif DataSet == 'DataSet30':
    AnalysisVariable.append('GENDERCODE')
    AnalysisVariable.append('AGEONADMISSION')
    AnalysisVariable.append('IndiginousFlag')
elif DataSet == 'DataSet40':
    AnalysisVariable.append('GENDERCODE')
    AnalysisVariable.append('age_range')
elif DataSet == 'DataSet60':
    AnalysisVariable.append('GENDERCODE')
    
       

importlib.reload(MLUtilities)
df = MLUtilities.setDefaultsLactate(df)

df_verification = df[AnalysisVariable].copy()
df_verification[DiagnosisString] = df_verification[DiagnosisString].astype('int')

df_verification['FirstLactateBG'] = df['FirstLactateBG'] 
df_verification['FirstBaseExcessBG'] =   df['FirstBaseExcessBG']
df_verification['FirstGCSScoreAdult'] = df['FirstGCSScoreAdult']


Emergency_IPInfo_Complete_Numeric[DiagnosisString] = Emergency_IPInfo_Complete_Numeric[DiagnosisString].astype('int')

df_verification['FirstBPSystolicRed'] = np.where(df_verification['FirstBPSystolic'] <=99, 1, np.where(df_verification['FirstBPSystolic'] >=180, 1, 0))
df_verification['FirstRespirationRed'] = np.where(df_verification['FirstRespiration'] <=7, 1, np.where(df_verification['FirstRespiration'] >=26, 1, 0))
df_verification['FirstSpO2Red'] = np.where(df_verification['FirstSpO2'] <=91, 1,  0)
df_verification['FirstO2FlowRed'] = np.where(df_verification['FirstO2Flow'] >=7, 1,  0)
df_verification['FirstPulseRateBPMRed'] = np.where(df_verification['FirstPulseRateBPM'] <=49, 1, np.where(df_verification['FirstPulseRateBPM'] >=120, 1, 0))
df_verification['FirstTemperatureDegreesCRed'] = np.where(df_verification['FirstTemperatureDegreesC'] <=35, 1, np.where(df_verification['FirstTemperatureDegreesC'] >=38.6, 1, 0))
df_verification['FirstLevelofConsciousnessRed'] = np.where(df_verification['FirstLevelofConsciousness'] >=2, 1,  0)
df_verification['FirstLactateBGRed'] = np.where(df_verification['FirstLactateBG'] >=2, 1,  0)

df_verification['FirstBPSystolicPurple'] = np.where(df_verification['FirstBPSystolic'] <=99, 1, np.where(df_verification['FirstBPSystolic'] >=200, 1, 0))
df_verification['FirstRespirationPurple'] = np.where(df_verification['FirstRespiration'] <=7, 1, np.where(df_verification['FirstRespiration'] >=31, 1, 0))
df_verification['FirstSpO2Purple'] = np.where(df_verification['FirstSpO2'] <=88, 1,  0)
df_verification['FirstO2FlowPurple'] = np.where(df_verification['FirstO2Flow'] >=8, 1,  0)
df_verification['FirstPulseRateBPMPurple'] = np.where(df_verification['FirstPulseRateBPM'] <=39, 1, np.where(df_verification['FirstPulseRateBPM'] >=140, 1, 0))
df_verification['FirstLevelofConsciousnessPurple'] = np.where(df_verification['FirstLevelofConsciousness'] >=3, 1,  0)
df_verification['FirstLactateBGPurple'] = np.where(df_verification['FirstLactateBG'] >=4, 1,  0)

df_verification['FirstRespirationqSOFA'] = np.where(df_verification['FirstRespiration'] >=22, 1, 0)

df_verification['FirstRespirationqSOFA'] = np.where(df_verification['FirstRespiration'] >=22, 1, 0)
df_verification['FirstBPSystolicqSOFA'] = np.where(df_verification['FirstBPSystolic'] <=100, 1, 0)
df_verification['FirstGCSScoreAdultqSOFA'] = np.where(df_verification['FirstGCSScoreAdult'] <15, 1, 0)

RedVariables = []
for c in df_verification.columns:
    if ('First' in c) & ('Red' in c): RedVariables.append(c)
        
PurpleVariables = []
for c in df_verification.columns:
    if ('First' in c) & ('Purple' in c): PurpleVariables.append(c)
    
    
qSOFAVariables = []
for c in df_verification.columns:
    if ('First' in c) & ('qSOFA' in c): qSOFAVariables.append(c)
    
df_verification['RedNumber'] = df_verification[RedVariables].sum(axis=1)
df_verification['PurpleNumber'] = df_verification[PurpleVariables].sum(axis=1)
df_verification['qSOFANumber'] = df_verification[qSOFAVariables].sum(axis=1)

y_preds_Purple = np.where(df_verification.PurpleNumber>0,1,0)
y_preds_Red = np.where(df_verification.RedNumber>1,1,0)
y_preds_qSOFA = np.where(df_verification.qSOFANumber>1,1,0)

df_verification['PurpleFlag'] = np.where(df_verification.PurpleNumber>0,True,False)
df_verification['RedFlag'] = np.where(df_verification.RedNumber>1,True,False)
df_verification['qSOFAFlag'] = np.where(df_verification.qSOFANumber>1,True,False)